In [2]:
import { z } from 'zod';
import { zodToJsonSchema } from 'zod-to-json-schema';
import { ChatOpenAI } from '@langchain/openai';
import { JsonOutputToolsParser } from '@langchain/core/output_parsers/openai_tools';
import {
  RunnableSequence,
  RunnableBranch,
  RunnablePassthrough,
} from '@langchain/core/runnables';
import { ChatPromptTemplate, PromptTemplate } from "@langchain/core/prompts";

const model = new ChatOpenAI({
  configuration: {
    baseURL: 'https://api.shellgpt.top/v1',
  },
  temperature: 0,
});

const classifySchema = z.object({
  type: z.enum(['科普', '编程', '一般问题']).describe('用户提问的分类'),
});

const modelWithTools = model.bind({
  tools: [
    {
      type: 'function',
      function: {
        name: 'classifyQuestion',
        description: '对用户的提问进行分类',
        parameters: zodToJsonSchema(classifySchema),
      },
    },
  ],
  tool_choice: {
    type: 'function',
    function: {
      name: 'classifyQuestion',
    },
  },
});

const prompt = ChatPromptTemplate.fromMessages([
  [
    'system',
    `仔细思考，你有充足的时间进行严谨的思考，然后对用户的问题进行分类，
    当你无法分类到特定分类时，可以分类到 "一般问题"`,
  ],
  ['human', '{input}'],
]);

const classifyChain = RunnableSequence.from([
  prompt,
  modelWithTools,
  new JsonOutputToolsParser(),
  (input) => {
    const type = input[0]?.args?.type;
    return type ? type : '一般问题';
  },
]);

await classifyChain.invoke({
  "input": "鲸鱼是哺乳动物么？"
})


"科普"

In [4]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const answeringModel = new ChatOpenAI({
    configuration: {
      baseURL: 'https://api.shellgpt.top/v1',
    },
    temperature: 0,
  });


const sciencePrompt = PromptTemplate.fromTemplate(
  `作为一位科普专家，你需要解答以下问题，尽可能提供详细、准确和易于理解的答案：

问题：{input}
答案：`
)

const programmingPrompt = PromptTemplate.fromTemplate(
  `作为一位编程专家，你需要解答以下编程相关的问题，尽可能提供详细、准确和实用的答案：

问题：{input}
答案：`
)

const generalPrompt = PromptTemplate.fromTemplate(
  `请回答以下一般性问题，尽可能提供全面和有深度的答案：

问题：{input}
答案：`
)


const scienceChain = RunnableSequence.from([
    sciencePrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "科普专家"
    }

])

const programmingChain = RunnableSequence.from([
    programmingPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "编程大师"
    }

])

const generalChain = RunnableSequence.from([
    generalPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "通识专家"
    }

])

In [5]:
const branch = RunnableBranch.from([
  [
    (input => input.type.includes("科普")),
    scienceChain,
  ],
  [
    (input => input.type.includes("编程")),
    programmingChain,
  ],
  generalChain
]);

In [8]:
const outputTemplate = PromptTemplate.fromTemplate(
  `感谢您的提问，这是来自 {role} 的专业回答：

  {output}
  `)


  const finalChain = RunnableSequence.from([
      {
          type: classifyChain,
          input: input => input.input
      },
      branch,
      (input) => {
        return outputTemplate.format(input)
      },
  ])

In [9]:
const res = await finalChain.invoke({
  "input": "鲸鱼是哺乳动物么？"
})

console.log(res)

input1122 : {
  output: "是的，鲸鱼是哺乳动物。尽管它们生活在水中，但鲸鱼属于哺乳动物类别。它们和其他哺乳动物一样，具有哺乳的特征，比如母鲸鱼会产奶喂养幼崽。此外，鲸鱼的身体结构也和其他哺乳动物相似，比如它们有呼吸器官和体毛。所以，尽管生活在水中，鲸鱼仍然是哺乳动物。",
  role: "科普专家"
}
感谢您的提问，这是来自 科普专家 的专业回答：

  是的，鲸鱼是哺乳动物。尽管它们生活在水中，但鲸鱼属于哺乳动物类别。它们和其他哺乳动物一样，具有哺乳的特征，比如母鲸鱼会产奶喂养幼崽。此外，鲸鱼的身体结构也和其他哺乳动物相似，比如它们有呼吸器官和体毛。所以，尽管生活在水中，鲸鱼仍然是哺乳动物。
  
